<H1>Demand Fairsharing in a Supply Chain Model</H1>

Recall the transportation problem of determining how many widgets to ship from each warehouse to each demand center in a transportation network, in order to satisfy customer demand at a minimum cost. To ensure that the model is always feasible, we can allow customer demand to go unsatisfied at a cost.

<H3> Sets and Indices </H3>

$i \in I$: Warehouses

$j \in J$: Customers (demand centers)

<H3>Data</H3>

$u_i$: capacity for warehouse $i$ (widgets)

$d_j$: demand at demand center $j$ (widgets)

$c_{ij}$: shipping cost from warehouse $i$ to customer $j$ ($/widget)

$\rho_j$: per unit demand shortfall cost for customer $j$ ($/widget)

<H3>Decision Variables</H3>

$x_{ij}$: number of widgets to ship from warehouse $i$ to customer $j$

$y_j$: demand shortfall for customer $j$

<H3>Linear Programming Formulation</H3>

\begin{eqnarray}
\min_{x} && \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} \;\; \mbox{(minimize shipping costs)} \nonumber \\
\mbox{s.t.} && \sum_{i \in I} x_{ij} = d_j - y_j,\;\;j \in J \;\; \mbox{(satisfy demand)}\nonumber \\
&& \sum_{j \in J} x_{ij} \le u_i,\;\;i \in I \;\; \mbox{(don't exceed capacity)} \nonumber \\
&& x_{ij} \ge 0, \;\;i \in I,\;j \in J \;\; \mbox{(ship nonnegative quantities)} \nonumber
\end{eqnarray}

<H3>Inputs</H3>
We'll consider a test data set with four demand centers that have the following demands:

In [ ]:
demands = [30, 36, 28, 40]
shortfall_costs = [1, 2, 1, 2]
customers = range(len(demands))

And with five warehouses that have the following widget capacities:

In [ ]:
capacities = [20, 22, 17, 19, 18]
warehouses = range(len(capacities))

We'll ignore shipping costs for now and focus on minimizing the demand shortfall. Assume meeting as much customer demand is top priority, so shipping costs can be handled as a secondary objective.

We can formulate and solve this model with Gurobi as follows.

In [ ]:
import gurobipy as grb

In [ ]:
model = grb.Model()

In [ ]:
to_ship = model.addVars(warehouses, customers, name='to_ship')

In [ ]:
shortfalls = model.addVars(customers, name='shortfall')

In [ ]:
model.update()

In [ ]:
demand_constrs = model.addConstrs((to_ship.sum('*', customer) == demands[customer] - shortfalls[customer]
                                  for customer in customers),
                                  name='demand')

In [ ]:
supply_constrs = model.addConstrs((to_ship.sum(warehouse, '*') <= capacities[warehouse]
                                  for warehouse in warehouses),
                                  name='supply')

In [ ]:
model.setObjective(shortfalls.prod(shortfall_costs))

In [ ]:
model.optimize()

In [ ]:
for customer, shortfall in shortfalls.items():
    print("Customer", customer, "has a demand shortfall of", shortfall.X, "units.")

With a shortfall cost structure like this, the model is motivated to meet as much demand as possible for the customers with the highest associated per unit shortfall costs. If we want to ensure a more fair distribution of demand shortfalls, we'll need to use a cost that is a convex function of the number of units shorted.

A quadratic function is a natural choice for such a cost function. In particular, the following quadratic cost functio n has the property that, under mild assumptions, each customer has the same percentage of their demand filled.

\begin{equation}
\sum_{j \in J} \frac{1}{d_j} y_j^2
\end{equation}

Exercise: Solve using the quadratic objective function. For each customer, report the demand shortfall as a percentage of total demand.